In [132]:
import matplotlib.pyplot as plt  # plotting library
import seaborn as sns  # plotting library
import math # math
import json
import glob
import csv
import statistics
import os
import numpy
from pathlib import Path
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
from pyvis import network as pvnet

# Heterogenous graphs representation

The graph contains two types of node:
* operation nodes;
* machine nodes.

Characteristics for operation node: id,

Characteristics for machine node: id.

Characteristics for operation ($O_p$) - operation ($O_s$) edge:

Characteristics for operation ($O_j$) - machine ($M_i$) edge: 
* setup_time - setup time needed for machine, $M_i$, to start the execution of operation, $O_j$
* execution_time - execution time needed for machine, $M_i$, to produce/assembly 1 unit product of operation, $O_j$
* actual_execution_time  - execution time that incorporets setup time for machime $M_i$ and execution time to produce/assembly the hole quantity of product need by operation $O_i$


In [133]:
%run BomInfo-Util.ipynb

In [134]:
%run BomInfo-OperationsGraph.ipynb import build_operation_tree

In [135]:
#graph with operations and machine nodes

def add_mahine_nodes_and_edges_in_heterogeneous_graph(current_node, G, product_machine_dictionary, machine_info={}, parent_quantity = None):
    operation_id = current_node['operationid']
    product_id = current_node['productid']

    if parent_quantity is None:
        quantity = 1
        parent_quantity = 1
    else:
        quantity = current_node['quantity']


    op_machines = product_machine_dictionary[product_id]
    

    for machine in op_machines:
        m_id = machine['id']
        actual_execution_time = machine['setup_time'] + machine['execution_time'] * quantity * parent_quantity
        #print("mi", operation_id, op_machines, machine_info)
        node_id = "M" + str(m_id)
     
        if node_id not in machine_info.keys():
            G.add_node(node_id)
            machine_info[node_id]= {"operations":[operation_id], 
                                    "setup_time":[machine['setup_time']],
                                    "execution_time": [machine['execution_time']],
                                    "actual_execution_time" : [actual_execution_time]
                                }
        else:
            machine_info[node_id]["operations"].append(operation_id)
            machine_info[node_id]["setup_time"].append(machine['setup_time'])
            machine_info[node_id]["execution_time"].append(machine['execution_time'])
            machine_info[node_id]["actual_execution_time" ].append(actual_execution_time)
                                
        G.add_edge(f"O{operation_id}", f"M{m_id}", 
                    setup_time=machine['setup_time'], 
                    execution_time=machine['execution_time'], 
                    actual_execution_time=actual_execution_time)

    if 'children' not in current_node:
        return 
    children = current_node['children']
    if len(children) == 0:
        return 
    for node in children:
        add_mahine_nodes_and_edges_in_heterogeneous_graph(node, G, product_machine_dictionary, 
                                                          machine_info, parent_quantity*quantity)


def add_machine_machine_edges(G, machine_info):
    ops = set()
    for m_id, m_info in machine_info.items():
        ops.update(machine_info[m_id]["operations"])
    ops_no = len(ops)
    
    m_ids =  list(machine_info.keys())
    for  i in range(len(m_ids)-1):
        m_i_ops = machine_info[m_ids[i]]['operations']
        for j in range(i+1, len(m_ids)):
            m_j_ops = machine_info[m_ids[j]]['operations']

            common_ops = set()
            common_ops.update(m_i_ops)
            common_ops.update(m_j_ops)
            #print(m_ids[i], m_ids[j])
            G.add_edge(m_ids[i], m_ids[j], shared_operations_percentage=len(common_ops)/ops_no)
    return ops_no
        
def add_machine_node_information(G, machine_info, ops_no):
    nodes_atts = {}
    for m_id, m_info in machine_info.items():
        nodes_atts[m_id]={}
        nodes_atts[m_id]["m_operations_percentage"] = len(m_info["operations"]) / ops_no
        nodes_atts[m_id]["m_mean_exec_time"] = mean(m_info["execution_time"])
        if len(m_info["execution_time"]) == 1:
            nodes_atts[m_id]["m_stdev_exec_time"] = 1
        else:
            nodes_atts[m_id]["m_stdev_exec_time"] = stdev(m_info["execution_time"])
        nodes_atts[m_id]["m_mean_setup_time"] = mean(m_info["setup_time"])
        if len(m_info["setup_time"]) == 1:
            nodes_atts[m_id]["m_stdev_setup_time"] = 1
        else:
            nodes_atts[m_id]["m_stdev_setup_time"] = stdev(m_info["setup_time"])
        nodes_atts[m_id]["m_mean_actual_exec_time"] = mean(m_info["actual_execution_time"])
        if len(m_info["actual_execution_time"]) == 1:
            nodes_atts[m_id]["m_stdev_actual_exec_time"] = 1
        else:
            nodes_atts[m_id]["m_stdev_actual_exec_time"] = stdev(m_info["actual_execution_time"])
        
    nx.set_node_attributes(G, nodes_atts)
    
def build_heterogeneuos_graph(current_node, G, product_machine_dictionary):
    """
    Node = operation
       - information = no. of machines on which can be produced
    Edge precedence relation betwwen operations(A,B) 
       - information = no. of B pices to produce A
    Edge (X,Y) edge between operations executed on same machine 
    """
    build_operation_tree(current_node, G, product_machine_dictionary)

    machine_info={}
    add_mahine_nodes_and_edges_in_heterogeneous_graph(current_node, G, product_machine_dictionary, machine_info=machine_info)

    ops_no = add_machine_machine_edges(G, machine_info)

    add_machine_node_information(G, machine_info, ops_no)
   


In [144]:
def build_heterogeneous_graph_charactristics(graphs):

    data_problem  = build_operation_graph_characteristics(graphs)

    data_problem['op_nodes_degree_assortativity_coefficient'] = []
    data_problem['m_nodes_degree_assortativity_coefficient'] = []
    
    for g in graphs:
        
        get_statistics(list(nx.get_edge_attributes(g, "setup_time").values()), data_problem, 'edge_setup_time')
        get_statistics(list(nx.get_edge_attributes(g, "execution_time").values()), data_problem, 'edge_execution_time')
        get_statistics(list(nx.get_edge_attributes(g, "actual_execution_time").values()), data_problem, 'edge_actual_execution_time')

        operations_nodes = [ (n) for n,e in g.nodes(data=True) if isinstance(n,str) and n[0] == 'O']
        machines_nodes = [ (n) for n,e in g.nodes(data=True) if isinstance(n,str) and n[0] == 'M']
        
        get_statistics([int(val) for (_, val) in g.degree(nbunch=operations_nodes)] , data_problem, 'node_operation_degree') 
        get_statistics([int(val) for (_, val) in g.degree(nbunch=machines_nodes)] , data_problem, 'node_machine_degree') 

        data_problem['op_nodes_degree_assortativity_coefficient'].append(nx.degree_assortativity_coefficient(g, nodes=operations_nodes))
        data_problem['m_nodes_degree_assortativity_coefficient'].append(nx.degree_assortativity_coefficient(g, nodes=machines_nodes))

        get_statistics(list(nx.get_node_attributes(g, "m_operations_percentage").values()), data_problem, 'node_m_operations_percentage')
        get_statistics(list(nx.get_node_attributes(g, "m_mean_exec_time").values()), data_problem, 'node_m_mean_exec_time')
        get_statistics(list(nx.get_node_attributes(g, "m_stdev_exec_time").values()), data_problem, 'node_m_stdev_exec_time')
        get_statistics(list(nx.get_node_attributes(g, "m_mean_setup_time").values()), data_problem, 'node_m_mean_setup_time')
        get_statistics(list(nx.get_node_attributes(g, "m_stdev_setup_time").values()), data_problem, 'node_m_stdev_setup_time')
        get_statistics(list(nx.get_node_attributes(g, "m_mean_actual_exec_time").values()), data_problem, 'node_m_mean_actual_exec_time')
        get_statistics(list(nx.get_node_attributes(g, "m_stdev_actual_exec_time").values()), data_problem, 'node_m_stdev_actual_exec_time')
   
    return data_problem
    

In [145]:
#CONFIGURATIONS
out_path_dir='../datasets/results/instances_characteristics/ianuarie2025-v2/heterogeneous'
dataset_path_dir='../datasets/'
weight_atts = [None, 'quantity', 's_execution_time', 's_quantity', 's_number', 
               'actual_execution_time'#o-m-edges
               ,'shared_operations_percentage']#m-m-edges


In [146]:
def buildDatasetHeterogeneous(dataset_graph, output_file_name):
    """
    :param: dataset_graph - list of heterogeneous multigraphs
    :param: output_file_name - file name to store characteristics
    """
    build_charactristics_info(dataset_graph, build_heterogeneous_graph_charactristics, 
                              out_path_dir, output_file_name, append_to_file=False, 
                              weight_atts=weight_atts)


In [147]:
def buildALL():
    """
    Build operation graph characteristics for all test  instances
    """
    #load like multigraphs
    _2asp_graphs, dyuthi_graphs, fjssp_graph, asp_deep_graphs, asp_wide_graphs, asp_mixed_graphs, dafjs_graphs, yfjs_graphs = load_graphs(build_heterogeneuos_graph)

    buildDatasetHeterogeneous(dyuthi_graphs, 'bom_info_graph_dyuthi.csv')
    # buildDatasetHeterogeneous(_2asp_graphs, 'bom_info_graph_2asp.csv')
    # buildDatasetHeterogeneous(fjssp_graph, 'bom_info_graph_fjssp.csv')
    # buildDatasetHeterogeneous(asp_wide_graphs, 'bom_info_graph_wide.csv')
    # buildDatasetHeterogeneous(dafjs_graphs, 'bom_info_graph_dafjs.csv')
    # buildDatasetHeterogeneous(yfjs_graphs, 'bom_info_graph_yfjs.csv')
    # buildDatasetHeterogeneous(asp_deep_graphs, 'bom_info_graph_deep.csv')
    # buildDatasetHeterogeneous(asp_mixed_graphs, 'bom_info_graph_mixed.csv')
   
buildALL()

out_file_name bom_info_graph_dyuthi.csv ../datasets/results/instances_characteristics/ianuarie2025-v2/heterogeneous
P1
	 None
	 quantity
	 s_execution_time
	 s_quantity
	 s_number
	 actual_execution_time
	 shared_operations_percentage
P10
	 None
	 quantity
	 s_execution_time
	 s_quantity
	 s_number
	 actual_execution_time
	 shared_operations_percentage
P11
	 None
	 quantity
	 s_execution_time
	 s_quantity
	 s_number
	 actual_execution_time
	 shared_operations_percentage
P12
	 None
	 quantity
	 s_execution_time
	 s_quantity
	 s_number
	 actual_execution_time
	 shared_operations_percentage
P13
	 None
	 quantity
	 s_execution_time
	 s_quantity
	 s_number
	 actual_execution_time
	 shared_operations_percentage
P14
	 None
	 quantity
	 s_execution_time
	 s_quantity
	 s_number
	 actual_execution_time
	 shared_operations_percentage
P15
	 None
	 quantity
	 s_execution_time
	 s_quantity
	 s_number
	 actual_execution_time
	 shared_operations_percentage
P16
	 None
	 quantity
	 s_execution_time
	 s_

In [148]:
_2asp_graphs, dyuthi_graphs, fjssp_graph, asp_deep_graphs, asp_wide_graphs, asp_mixed_graphs, dafjs_graphs, yfjs_graphs = load_graphs(build_heterogeneuos_graph)

    #buildDatasetHeterogeneous(_2asp_graphs, 'bom_info_graph_2asp.csv')
    #buildDatasetHeterogeneous(fjssp_graph, 'bom_info_graph_fjssp.csv')
    #bom_fjssp_1_7_7_60_10_No1

buildDatasetHeterogeneous(dyuthi_graphs, 'bom_info_graph_dyuthi.csv')
 

out_file_name bom_info_graph_dyuthi.csv ../datasets/results/instances_characteristics/ianuarie2025-v2/heterogeneous
P1
	 None
	 quantity
	 s_execution_time
	 s_quantity
	 s_number
	 actual_execution_time
	 shared_operations_percentage
P10
	 None
	 quantity
	 s_execution_time
	 s_quantity
	 s_number
	 actual_execution_time
	 shared_operations_percentage
P11
	 None
	 quantity
	 s_execution_time
	 s_quantity
	 s_number
	 actual_execution_time
	 shared_operations_percentage
P12
	 None
	 quantity
	 s_execution_time
	 s_quantity
	 s_number
	 actual_execution_time
	 shared_operations_percentage
P13
	 None
	 quantity
	 s_execution_time
	 s_quantity
	 s_number
	 actual_execution_time
	 shared_operations_percentage
P14
	 None
	 quantity
	 s_execution_time
	 s_quantity
	 s_number
	 actual_execution_time
	 shared_operations_percentage
P15
	 None
	 quantity
	 s_execution_time
	 s_quantity
	 s_number
	 actual_execution_time
	 shared_operations_percentage
P16
	 None
	 quantity
	 s_execution_time
	 s_

# Visualisation

In [37]:
print(dyuthi_graphs)

[<networkx.classes.multigraph.MultiGraph object at 0x7fcb6e7f9c30>, <networkx.classes.multigraph.MultiGraph object at 0x7fcb6e7ac3d0>, <networkx.classes.multigraph.MultiGraph object at 0x7fcb6e77d180>, <networkx.classes.multigraph.MultiGraph object at 0x7fcb6e7ccb50>, <networkx.classes.multigraph.MultiGraph object at 0x7fcb95980a60>, <networkx.classes.multigraph.MultiGraph object at 0x7fcb7369f7f0>, <networkx.classes.multigraph.MultiGraph object at 0x7fcb7369df00>, <networkx.classes.multigraph.MultiGraph object at 0x7fcb7369e890>, <networkx.classes.multigraph.MultiGraph object at 0x7fcb7369ef20>, <networkx.classes.multigraph.MultiGraph object at 0x7fcb7369dff0>, <networkx.classes.multigraph.MultiGraph object at 0x7fcb7369e170>]


In [99]:
#display

def plot_g_pyviz(G, name='out.html', height='300px', width='700px'):
   
    net = pvnet.Network(notebook=True, directed=True, height=height, width=width)
    opts = '''
        var options = {
          "physics": {
            "forceAtlas2Based": {
              "gravitationalConstant": -100,
              "centralGravity": 0.11,
              "springLength": 100,
              "springConstant": 0.09,
              "avoidOverlap": 1
            },
            "minVelocity": 0.75,
            "solver": "forceAtlas2Based",
            "timestep": 0.22
          }
        }
    '''

    
    net.set_options(opts)
    # uncomment this to play with layout
    # net.show_buttons(filter_=['physics'])
    
    g = G.copy() # some attributes added to nodes
    net.from_nx(g)
    for edge in net.edges:
         #print(edge)
        if 'machine' in edge and isinstance(edge['machine'],str) and edge['machine'][0] == 'M':
            edge['color']='red'
       
    for node in net.nodes:
        #print(node)
        if isinstance(node['label'],str) and node['label'][0] == 'M':
            node['color']='red'
            node['shape']='box'
        #print(node)
    return net.show(name)

In [102]:
plot_g_pyviz(dyuthi_graphs[0])

NameError: name 'dyuthi_graphs' is not defined